## Introduction

### Bing Search API:
The Bing Search API is a powerful tool provided by Microsoft, enabling developers to integrate Bing's search capabilities directly into their applications. This API allows access to a wide range of search results, including web pages, images, videos, and news, providing developers with a seamless way to incorporate Bing's search functionality into their own projects.

### Transform Chain in LangChain:
**Langchain** is a Python library that allows you to create powerful applications using language models and prompts.
Langchain's **Transform Chain** is a dynamic and versatile feature that facilitates the seamless transformation of text data. Through a series of customizable linguistic operations, developers can construct intricate text processing pipelines. This transformative chain empowers users to preprocess, analyze, and manipulate textual information efficiently, offering a flexible and scalable solution for diverse language processing tasks within the Langchain framework.

## Objective
1. To setup a request to call Bing API
2. To parse the Bing search response for Bio
3. To setup Transform chain and execute for grounding the Bio

### Import Libararies

In [7]:
from langchain.chains import TransformChain
import requests
import json
from openai import AzureOpenAI

### Get Environment variables

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")


Found Azure OpenAI API Base Endpoint: https://swed-oai.openai.azure.com/


In [3]:
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"
bing_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

### Get Guest name from file

In [4]:
# Open the file
with open('guest_name.txt', 'r') as f:
    guest = f.read()

# Print the content
print(guest)

Neil deGrasse Tyson


## 1. Test Bing Search 

### Define function to perform Bing search

In [8]:
def search_bing(query):
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)

### Test Bing search and evaluate the search results

In [9]:
search_bing("Who is "+guest+"?")

'[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson | Biography, Books, TV Shows, & Facts", "link": "https://www.britannica.com/biography/Neil-deGrasse-Tyson", "snippet": "Neil deGrasse Tyson (born October 5, 1958, New York, New York, U.S.) American astronomer who popularized science with his books and frequent appearances on radio and television. (Read Neil deGrasse Tyson\\u2019s Britannica essay on public science.)"}, {"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://nl.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse 

## 2. Test GPT model without grounding

### Create a prompt for GPT model

In [10]:
#messages = [{"role": "user", "content": "Provide a very short biography of "+guest + " including lates events"}]
messages = [{"role": "user", "content": "Who is "+guest+"?"}]

### Connect to Azure OpenAI service

In [13]:
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-12-01-preview"
)
model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

gpt-4


### Send request to GPT model

In [14]:
gpt_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  
    
bio = gpt_response.choices[0].message.content
print(bio)

Neil deGrasse Tyson is an American astrophysicist, cosmologist, author, and science communicator. He is best known for his role as the host of the television show "Cosmos: A Spacetime Odyssey," where he presented scientific concepts in an understandable and exciting way to the general public. Tyson also serves as the director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City. In addition, he is a prolific author who has written numerous books and papers about space and science. Tyson's passion and enthusiasm for science has made him a popular figure among both scientists and science enthusiasts. He's known for his ability to explain complex scientific concepts in a clear and entertaining way.



## 3. Grounding with Function Calling

### Define Tools for GPT model, that include Bing search function

In [15]:
#messages = [{"role": "user", "content": "Provide the latest information about "+guest}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information about people",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

### Send request to GPT model and see if there any tools calling

In [16]:
response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        )

response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lWEUb2sV608onDmfoPlr6OSd', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson"\n}', name='search_bing'), type='function')])
[ChatCompletionMessageToolCall(id='call_lWEUb2sV608onDmfoPlr6OSd', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson"\n}', name='search_bing'), type='function')]


### Go through function calling and extract messages to see Bing search results

In [17]:
if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_bing": search_bing,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
print(messages)

[{'role': 'user', 'content': 'Who is Neil deGrasse Tyson?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lWEUb2sV608onDmfoPlr6OSd', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson"\n}', name='search_bing'), type='function')]), {'tool_call_id': 'call_lWEUb2sV608onDmfoPlr6OSd', 'role': 'tool', 'name': 'search_bing', 'content': '[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://nl.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snip

### Ask GPT model to generate short bio of the person based on search results

In [18]:
second_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response
    
bio = second_response.choices[0].message.content
print(bio)

Neil deGrasse Tyson is an American astrophysicist, author, and science communicator. Born on October 5, 1958, he has studied at Harvard University, the University of Texas at Austin, and Columbia University. Tyson has made a significant impact on the field of astronomy by popularizing science through his books and frequent appearances on radio and television. He has hosted TV series such as Origins and NOVA scienceNOW, as well as the radio show StarTalk. One of his noted books is "The Pluto Files: The Rise and Fall of America's Favorite Planet". Tyson is also the director of the Hayden Planetarium and a board member of The Planetary Society.


### Compare with non grounded answer
Neil deGrasse Tyson is an American astrophysicist, cosmologist, author, and science communicator. He is best known for his role as the host of the television show "Cosmos: A Spacetime Odyssey," where he presented scientific concepts in an understandable and exciting way to the general public. Tyson also serves as the director of the Hayden Planetarium at the Rose Center for Earth and Space in New York City. In addition, he is a prolific author who has written numerous books and papers about space and science. Tyson's passion and enthusiasm for science has made him a popular figure among both scientists and science enthusiasts. He's known for his ability to explain complex scientific concepts in a clear and entertaining way.



### Save the biography to file

In [ ]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)

## 4. Grounding with LangChain

In [ ]:
def bing_grounding(input_dict:dict) -> dict:
    print("Calling Bing Search API to get bio for guest...\n")
    search_term = input_dict["guest"]
    print("Search term is " + search_term)

    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    #print(search_results)

    # Parse out a bio.  
    bio = search_results["webPages"]["value"][0]["snippet"]
    
    print("Bio:\n")
    print(bio)
    print("\n")

    return {"bio": bio}


In [ ]:
bing_chain = TransformChain(input_variables=["guest"], output_variables=["bio"], transform=bing_grounding)
bio = bing_chain.run(guest)


### Save biography to file

In [ ]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)